In [1]:
import pandas
import uproot
import matplotlib.pyplot as plt
import numpy as np
import scipy.interpolate as interpolate
import pickle

from uncertainties import ufloat

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Roman']})
rc('text', usetex=True)
params= {'text.latex.preamble' : [r'\usepackage{amsmath}']}
plt.rcParams.update(params)
plt.rcParams['text.latex.preamble'] = r"\usepackage{bm} \usepackage{amsmath}"

/cvmfs/sft.cern.ch/lcg/releases/Python/3.8.6-3199b/x86_64-centos7-gcc8-opt/lib/python3.8/_collections_abc.py:832: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [2]:
#Dowloading the data
bc_tree = uproot.open('/eos/experiment/fcc/ee/analyses/case-studies/flavour/Bc2TauNu/flatNtuples/spring2021/prod_04/Analysis_stage2/p8_ee_Zbb_ecm91_EvtGen_Bc2TauNuTAUHADNU.root')['events']
bu_tree = uproot.open('/eos/experiment/fcc/ee/analyses/case-studies/flavour/Bc2TauNu/flatNtuples/spring2021/prod_04/Analysis_stage2/p8_ee_Zbb_ecm91_EvtGen_Bu2TauNuTAUHADNU.root')['events']
bb_tree = uproot.open('/eos/experiment/fcc/ee/analyses/case-studies/flavour/Bc2TauNu/flatNtuples/spring2021/prod_04/Analysis_stage2/p8_ee_Zbb_ecm91_EvtGen.root')['events']
cc_tree = uproot.open('/eos/experiment/fcc/ee/analyses/case-studies/flavour/Bc2TauNu/flatNtuples/spring2021/prod_04/Analysis_stage2/p8_ee_Zcc_ecm91.root')['events']

tree_gen = uproot.open("/eos/experiment/fcc/ee/analyses/case-studies/flavour/Bc2TauNu/flatNtuples/spring2021/prod_04/Analysis_stage2/p8_ee_Zbb_ecm91_EvtGen_Bc2TauNuTAUHADNU.root")["metadata"]

### General parameters

In [ ]:
#the for theoretical
#sim for simulated
#exp for experimental

#Number of Z0 produced
N_Z0_init = 5*10**(12)
N_Z0 = 5*10**(11)

#Branching fraction Z->quark-pair
exp_BF_Zbb = 0.1512
err_BF_Zbb = 0.0005

BF_Zbb = ufloat(exp_BF_Zbb, err_BF_Zbb)

#Production ratio of Bc+ and B+
sim_F_Bc = 0.0004 #pythia simulations
err_F_Bc = 

F_Bc = ufloat(sim_F_Bc, err_F_Bc)

#Branching fraction and errors for (Bc+ -> tau nu)
the_BF_Bctaunu = 0.0194
err_BF_Bctaunu = 

BF_Bctaunu = ufloat(the_BF_Bctaunu, err_BF_Bctaunu)

#Branching fraction and errors for (tau -> 3pions nu)
exp_BF_tau3pion = 0.0931
err_BF_tau3pion = 0.0005

BF_tau3pion = ufloat(exp_BF_tau3pion, err_BF_tau3pion)

#Branching fraction and errors for (Bc+ -> J/psi tau nu) 
the_BF_BcJpsitaunu = 0.0135
err_BF_BcJpsitaunu = 0.0011

BF_BcJpsitaunu = ufloat(the_BF_BcJpsitaunu, err_BF_BcJpsitaunu)

#Branching fraction and errors for (J/psi -> mu mu)
exp_BF_Jpsimumu = 0.05961
err_BF_Jpsimumu = 0.00033

BF_Jpsimumu = ufloat(exp_BF_Jpsimumu, err_BF_Jpsimumu)

#Cut efficiencies for the optimised cut
epsilon_Bctaunu = 
epsilon_BcJpsitaunu = 0.1

#Yields have to be computed

### Signal yields for different number of $Z^0$ bosons

In [ ]:
#5e12 Z0
sig_5e12 = 4009.2649482816005
err_sig_5e12 = 125.2
uncertainty_sig_5e12 = ufloat(sig_5e12, err_sig_5e12)

#4e12 Z0
sig_4e12 = 3207.4119586252805
err_sig_4e12 = 110.6
uncertainty_sig_4e12 = ufloat(sig_4e12, err_sig_4e12)

#3e12 Z0
sig_3e12 = 2405.55896896896
err_sig_3e12 = 98.76
uncertainty_sig_3e12 = ufloat(sig_3e12, err_sig_3e12)

#2e12 Z0
sig_2e12 = 1603.7059793126402
err_sig_2e12 = 80.78
uncertainty_sig_2e12 = ufloat(sig_2e12, err_sig_2e12)

#1e12 Z0
sig_1e12 = 801.8529896563201
err_sig_1e12 = 57.34
uncertainty_sig_1e12 = ufloat(sig_1e12, err_sig_1e12)

#0.5e12 Z0
sig_5e11 = 400.92649482816006
err_sig_5e11 = 40.26
uncertainty_sig_5e11 = ufloat(sig_5e11, err_sig_5e11)

In [ ]:
#Creating the associated arrays
SIG = np.array([sig_5e11, sig_1e12, sig_2e12, sig_3e12, sig_4e12, sig_5e12])
ERR_SIG = np.array([err_sig_5e11, err_sig_1e12, err_sig_2e12, err_sig_3e12, err_sig_4e12, err_sig_5e12])

sig_yield_5e12 = 4010
sig_yield_4e12 = 3214
sig_yield_3e12 = 2403
sig_yield_2e12 = 1607
sig_yield_1e12 = 804
sig_yield_5e11 = 399.2

SIG = np.array([sig_yield_5e11, sig_yield_1e12, sig_yield_2e12, sig_yield_3e12, sig_yield_4e12, sig_yield_5e12])

### Normalisation channel yields for different number of $Z^0$ bosons

In [ ]:
def norm(N_Z0) :
    return N_Z0*BF_Zbb*2*F_Bc*BF_BcJpsitaunu*epsilon_Bctaunu

#Est ce que je dois rajouter J/Psi -> mu mu ?

In [ ]:
#5e12 Z0
norm_5e12 = 4009.2649482816005
err_sig_5e12 = 125.2
uncertainty_sig_5e12 = ufloat(sig_5e12, err_sig_5e12)

#4e12 Z0
norm_4e12 = 3207.4119586252805
err_sig_4e12 = 110.6
uncertainty_sig_4e12 = ufloat(sig_4e12, err_sig_4e12)

#3e12 Z0
norm_3e12 = 2405.55896896896
err_sig_3e12 = 98.76
uncertainty_sig_3e12 = ufloat(sig_3e12, err_sig_3e12)

#2e12 Z0
norm_2e12 = 1603.7059793126402
err_sig_2e12 = 80.78
uncertainty_sig_2e12 = ufloat(sig_2e12, err_sig_2e12)

#1e12 Z0
norm_1e12 = 801.8529896563201
err_sig_1e12 = 57.34
uncertainty_sig_1e12 = ufloat(sig_1e12, err_sig_1e12)

#0.5e12 Z0
norm_5e11 = 400.92649482816006
err_norm_5e11 = 40.26
uncertainty_sig_5e11 = ufloat(sig_5e11, err_sig_5e11)